In [ ]:
import pandas as pd
import geopandas
import numpy as np
import os
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import geopandas as gpd
import pycountry
import descartes
import contextily as ctx
from shapely.geometry import Point, Polygon

In [ ]:
df_fires = pd.read_csv("./dataset_incendios.csv", delimiter=";", encoding="iso-8859-1")
df_fires['departamento_id_fixed'] = [int(prov_id + dept_id) for prov_id, dept_id in zip([str(x) for x in df_fires['provincia_id']], [str(x).zfill(3) for x in df_fires['departamento_id']])]

df_provinces = pd.read_csv("./provincias.csv", delimiter=",")
df_provinces.rename(columns={'centroide_lat': 'prov_lat', 'centroide_lon': 'prov_long', 'nombre': 'nombre_provincia', 'id': 'id_provincia'}, inplace=True)
new_cols = list(set(df_provinces.columns) - set(['categoria', 'fuente', 'iso_nombre', 'nombre_completo']))
df_provinces = df_provinces[new_cols]

df_departments = pd.read_csv("./departamentos.csv", delimiter=",")
df_departments.rename(columns={'centroide_lat': 'dept_lat', 'centroide_lon': 'dept_long', 'nombre': 'nombre_departamento', 'id': 'id_departamento'}, inplace=True)
new_cols = list(set(df_departments.columns) - set(['categoria', 'fuente', 'nombre_completo', 'provincia_id', 'provincia_interseccion', 'provincia_nombre']))
df_departments = df_departments[new_cols]


df = pd.merge(df_fires, df_provinces, left_on='provincia_id', right_on='id_provincia')
df = pd.merge(df, df_departments, left_on='departamento_id_fixed', right_on='id_departamento')

df['departamento_point'] = [Point(xy) for xy in zip(df['dept_long'], df['dept_lat'])]
df['provincia_point'] = [Point(xy) for xy in zip(df['prov_long'], df['prov_lat'])]
new_cols = list(set(df.columns) - set(['provincia', 'departamento_id', 'departamento', 'id_provincia', 'id_departamento', 'dept_lat', 'dept_long', 'prov_long', 'prov_lat', 'pais_id', 'iso_id']))
df = df[new_cols]
df['provincia_id'] = [str(x).zfill(2) for x in df['provincia_id']]
df['departamento_id_fixed'] = [str(x).zfill(5) for x in df['departamento_id_fixed']]

In [ ]:
geo_provinces = gpd.read_file('./provincias/provincias.shp')
geo_departments = gpd.read_file('./departamentos/departamentos.shp')

In [ ]:
#fires_by_province = df.groupby("provincia_id").size().reset_index(name='counts')
fires_by_province = df.groupby("provincia_id")['cant_focos'].agg('sum').reset_index(name='counts')
geo_provinces_to_plot = pd.merge(geo_provinces, fires_by_province, left_on='IN1', right_on='provincia_id', how='outer')
geo_provinces_to_plot = geo_provinces_to_plot[list(['NAM', 'IN1', 'geometry', 'counts'])].fillna(0)
geo_provinces_to_plot['rank'] = geo_provinces_to_plot['counts'].rank()

In [ ]:
#fires_by_departments = df.groupby("departamento_id_fixed").size().reset_index(name='counts')
#geo_departaments_to_plot = pd.merge(geo_departments, fires_by_departments, left_on='IN1', right_on='departamento_id_fixed', how='outer')
#geo_departaments_to_plot = geo_departaments_to_plot[list(['NAM', 'IN1', 'geometry', 'counts'])].fillna(0)
#geo_departaments_to_plot['rank'] = geo_departaments_to_plot['counts'].rank()

In [ ]:
geo_departaments_to_plot = pd.merge(geo_departments, df, left_on='IN1', right_on='departamento_id_fixed', how='inner')
geo_departaments_to_plot = geo_departaments_to_plot[list(['NAM', 'IN1', 'departamento_point'])]
geo_departaments_to_plot = gpd.GeoDataFrame(geo_departaments_to_plot, geometry=df.departamento_point)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 20))
ax.set_aspect('equal')
ax.set_xlim(-74.0, -50.0)
ax.set_ylim(-57.0, -21.0)
ax.set_xticks([])
ax.set_yticks([])
ax.set_title("Incendios forestales por provincia", size=16)
geo_provinces_to_plot.plot(column='rank', cmap='Reds', linewidth=0.8, ax=ax, edgecolor='0.8')
geo_departaments_to_plot.plot(ax=ax, marker='o', color='blue', markersize=10); sns.set()
plt.box(on=None)